In [1]:
import pandas as pd
from pysam import FastaFile
import json
import logging
from crc64iso import crc64iso

In [2]:
logger = logging.getLogger(__name__)

In [3]:
protein_fa = FastaFile('/Users/liang/Box/MyCPTAC/CPTAC_proteome_v1/DCC/RefSeq_20111201/20111201_RefSeq_Human_37-Mouse_37_Trypsin.renamed.fasta.gz')

In [4]:
with open('../../intermediates/refseq_20111201/refseq_protein_ids.list') as f:
    refseq_ids = f.read().splitlines()

len(refseq_ids)

32799

In [5]:
prot_id = refseq_ids[0]
prot_seq = protein_fa.fetch(prot_id)

In [6]:
df = pd.DataFrame({'original_protein_id': refseq_ids})
df['uniparc_checksum'] = df['original_protein_id'].apply(protein_fa.fetch).apply(crc64iso.crc64)

In [7]:
df.to_csv('../../intermediates/refseq_20111201/refseq_crc64_checksums.tsv', sep='\t', index=False)

In [8]:
with open('../../intermediates/refseq_20111201/uniparc_aria2_downloads.links', 'w') as f:
    for t in df.itertuples(index=False):
        print(
            f'https://www.uniprot.org/uniparc/?query=checksum%3A{t.uniparc_checksum}&format=xml\n'
            f'  out=xmls/{t.original_protein_id}.xml',
            file=f
        )